Roger Mei

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
path = "./DillardsPOS/"
data_str = pd.read_csv(path + "strinfo.csv", header=None,names = ["STORE_PK","CITY","STATE","ZIP","binary"],dtype={"STORE_PK": int, "CITY": str,"STATE":str,"ZIP":str})
data_str = data_str.drop(columns=["binary"])

data_skst = pd.read_csv(path + "skstinfo.csv", header=None,names = ["SKU_PK","STORE_PK","COST","RETAIL","binary"],dtype={"SKU_PK": int,"STORE_PK": int, "COST":float,"RETAIL":float})
data_skst = data_skst.drop(columns=["binary"])

data_sku = pd.read_csv(path + "skuinfo.csv", header=None,warn_bad_lines=False,error_bad_lines=False,names = ["SKU_PK","DEPT_FK","CLASSID","UPC","STYLE","COLOR","SIZE","PACKSIZE","VENDOR","BRAND","binary"])
data_sku = data_sku.drop(columns=["binary"])

data_dept = pd.read_csv(path + "deptinfo.csv", header=None,names = ["DEPT_PK","DEPTDESC","binary"],dtype={"DEPT_PK":int,"DEPTDESC":str,"binary":int})
data_dept = data_dept.drop(columns=["binary"])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
path = "./DillardsPOS/"
data_trnsact = pd.read_csv(path + "trnsact.csv", header=None,names = ["SKU","STORE","REGISTER","TRANNUM","SEQ","SALEDATE","STYPE","QUANTITY","price1","price2","price3","INTERID","MIC","binary"],dtype={"SKU": int,"STORE": int,"REGISTER":int,"TRANNUM":int,"SEQ":int,"SALEDATE":str,"STYPE":str,"QUANTITY":int,"price1":float,"price2":float,"price3":float,"INTERID":int,"MIC":int,"binary":int})
data_trnsact = data_trnsact.drop(columns=["price3"]) #duplicate column of price2
data_trnsact = data_trnsact.drop(columns=["binary"])

# Data exploration and previewing tables:

In [26]:
#STRINFO table
print("number of stores: ", data_str.shape[0])
data_str.tail()

number of stores:  453


,STORE_PK,CITY,STATE,ZIP
448,9808,GILBERT,AZ,85233
449,9812,METAIRIE,LA,70006
450,9900,LITTLE ROCK,AR,72201
451,9906,LITTLE ROCK,AR,72201
452,9909,CHEYENNE,WY,82009


In [4]:
#SKSTINFO table
print("number of sku stores: ", data_skst.shape[0])
data_skst.tail()

number of sku stores:  39230146


,SKU_PK,STORE_PK,COST,RETAIL
39230141,9999997,2007,15.0,19.50
39230142,9999997,2707,15.0,9.75
39230143,9999997,3307,15.0,19.50
39230144,9999997,7507,15.0,19.50
39230145,9999997,7907,15.0,19.50


In [5]:
#TRNSACT table
print("number of transactions: ", data_trnsact.shape[0])
data_trnsact.tail()

number of transactions:  120916896


,SKU,STORE,REGISTER,TRANNUM,SEQ,SALEDATE,STYPE,QUANTITY,price1,price2,INTERID,MIC
120916891,9999997,7507,750,2800,0,2005-04-23,P,1,39.0,39.00,593900116,395
120916892,9999997,7907,790,3200,0,2005-05-03,P,1,39.0,39.00,502600061,395
120916893,9999997,7907,800,700,0,2005-05-04,R,1,39.0,39.00,600900066,395
120916894,9999997,7907,800,700,0,2005-05-04,P,1,39.0,39.00,601000066,395
120916895,9999997,7907,800,4100,257808914,2005-07-31,P,1,39.0,26.13,658600082,395


Testing composite key in transaction table:

In [14]:
test1 = data_trnsact[data_trnsact.TRANNUM == 2800]
test1 = test1[test1.SALEDATE == "2005-06-28"]
test1 = test1[test1.STORE == 3902]
test1 = test1[test1.REGISTER == 390]
test1.head()

,SKU,STORE,REGISTER,TRANNUM,SEQ,SALEDATE,STYPE,QUANTITY,price1,price2,INTERID,MIC
3730294,308483,3902,390,2800,648708895,2005-06-28,P,1,69.0,49.99,457800049,764


In [22]:
#SKUINFO table
print("number of SKUs: ", data_sku.shape[0])
data_sku.tail()

number of SKUs:  1564178


,SKU_PK,DEPT_FK,CLASSID,UPC,STYLE,COLOR,SIZE,PACKSIZE,VENDOR,BRAND
1564173,9999973,3103,009,400009973999,702 S3JAYV,STONE,4,1,6813115,POLO JEA
1564174,9999974,9801,726,400009974999,G50171,NAVY MULTI,10,1,9212766,GABAR IN
1564175,9999991,2301,004,400009991999,026 MDU201,618RED ROSE,8,1,0023272,JONES/LA
1564176,9999992,1202,402,400009992999,14 F52UN1,PALE JADE,L,1,1446212,CABERNET
1564177,9999997,2503,111,400009997999,1 1XKBGO,210CHOPNK,7,1,8515392,RAMPAGE


In [256]:
#DEPTINFO table
print("number of departments: ", data_dept.shape[0])
data_dept.head(10)

number of departments:  60


,DEPT_PK,DEPTDESC
0,800,CLINIQUE
1,801,LESLIE
2,1100,GARY F
3,1107,JACQUES
4,1202,CABERN
5,1301,BE2
6,1704,R LAUREN
7,1905,R & Y
8,2102,CAB
9,2105,R TAYLOR


Polo Jeans Company, LLC is a subsidiary of the Ralph Lauren Corporation.
We look at only the brand Polo Jeans Co. and clothing in the department Ralph Lauren to see if clothing by different brands but part of the same company would have any association. 

In [4]:
# POLO JEANS CO. brand
sku_polo = data_sku[data_sku.BRAND == "POLO JEA "]
polo = list(sku_polo.SKU_PK)
# RALPH LAUREN dept
sku_rla = data_sku[data_sku.DEPT_FK == 1704]
rlauren = list(sku_rla.SKU_PK)
prl = polo + rlauren
len(prl)

22856

In [29]:
print("number of items with brand POLO JEANS CO.: ",len(polo))
print("number of items in dept RALPH LAUREN: ",len(prl))

number of items with brand POLO JEANS CO.:  7261
number of items in dept RALPH LAUREN:  22856


In [33]:
set(sku_polo.DEPT_FK).__and__(set(rlauren))

set()

There is no overlap between the set of SKUs in the Ralph Lauren dept and the set of SKUs with the Polo Jeans Co. brand. This means that all items of the brand Polo Jeans Co. are not located in the Ralph Lauren dept. If there are associations, these items should be in the same department or at least close to each other in the store.

Looking at number of transactions per year, just to see how spread out the data is in time:

In [7]:
data_trnsact[data_trnsact['SALEDATE'].str.contains("2003")].shape[0]

0

In [4]:
data_trnsact[data_trnsact['SALEDATE'].str.contains("2004")].shape[0]

46103710

In [5]:
data_trnsact[data_trnsact['SALEDATE'].str.contains("2005")].shape[0]

74813186

In [6]:
data_trnsact[data_trnsact['SALEDATE'].str.contains("2006")].shape[0]

0

# Filtering dataset:

Looking at IL stores:

In [6]:
str_il = data_str[data_str.STATE == "IL"]
str_il

,STORE_PK,CITY,STATE,ZIP
42,603,FAIRVIEW HEIGHTS,IL,62208
64,1003,MARION,IL,62959
251,4903,MOLINE,IL,61265


In [37]:
print("number of transactions per IL store:")
print("store 603: ",sum(data_trnsact.STORE == 603))
print("store 1003: ",sum(data_trnsact.STORE == 1003))
print("store 4903: ",sum(data_trnsact.STORE == 4903))
print("total:")
print(sum(data_trnsact.STORE == 603) + sum(data_trnsact.STORE == 1003) + sum(data_trnsact.STORE == 4903))

number of transactions per IL store:
store 603:  309095
store 1003:  292294
store 4903:  145456
total:
746845


In the future, we might want to build a new store in Chicago. Performing analysis on IL stores may also provide business insights to a new Chicago retail location.

In [5]:
trnsact_il = data_trnsact[data_trnsact.STORE.isin([603,1003,4903])]

Looking at only purchase transaction types:

In [6]:
trnsact_il = trnsact_il[trnsact_il.STYPE == "P"]
trnsact_il.shape[0]

683509

Looking at only the brand Polo Jeans Co. and clothing in the department Ralph Lauren:

In [7]:
trnsact_il = trnsact_il[trnsact_il['SKU'].isin(prl)]

Because transaction types are the same, we drop the STYPE column:

In [8]:
trnsact_il = trnsact_il.drop(columns=["STYPE"])

# Combinining Transaction table primary keys into one unique transaction identifier

In [9]:
trnsact_il = trnsact_il.assign(id=(trnsact_il['SALEDATE'] + '_' 
                                      + trnsact_il['REGISTER'].astype(str) + '_' 
                                      + trnsact_il['TRANNUM'].astype(str) + '_'
                                      + trnsact_il['STORE'].astype(str) + '_')
                                  .astype('category').cat.codes)

Since they are already combined into a single identifier, we drop the 4 primary key columns used:

In [10]:
trnsact_il = trnsact_il.drop(columns=["REGISTER","TRANNUM","SALEDATE","STORE"])

Dropping unused columns as well:

In [11]:
trnsact_il = trnsact_il.drop(columns=["SEQ","price1","price2","INTERID","MIC"])

Looking at the counts of each composite key, the count of each key seems to make sense as it does not get too large. They stay within reasonable values as what the number of items in a single transaction should be.

In [34]:
trnsact_il.id.value_counts()[:20]

317     14
1831    12
856     11
1873     8
321      8
305      8
598      8
2673     8
1911     8
230      7
3437     7
600      7
351      7
1399     7
322      7
1844     7
490      7
3341     7
621      7
2581     6
Name: id, dtype: int64

3672 transactions are of Polo Jeans Co. brand and/or are items in the Ralph Lauren department.

In [339]:
trnsact_il.head()

,SKU,QUANTITY,id
20939,4072,1,860
20979,4072,1,1052
20980,4072,1,1046
20981,4072,1,1900
145725,11563,1,150


In [336]:
trnsact_il.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5475 entries, 20939 to 120825016
Data columns (total 3 columns):
SKU         5475 non-null int64
QUANTITY    5475 non-null int64
id          5475 non-null int16
dtypes: int16(1), int64(2)
memory usage: 139.0 KB


# SKU supports:

Frequency of each specified item in IL:

In [338]:
trnsact_il.SKU.value_counts(normalize=True)

4747172    0.003653
5760331    0.003288
5540331    0.003288
5770331    0.003288
1726624    0.002922
4707172    0.002922
7604275    0.002557
6974275    0.002374
6842459    0.002374
9586806    0.002192
3082459    0.002192
3092459    0.002192
6474085    0.002192
4687172    0.002009
5332459    0.002009
3022459    0.002009
6874275    0.002009
7072459    0.001826
8784749    0.001826
1314861    0.001826
6884861    0.001826
5584721    0.001826
1736624    0.001826
5750331    0.001826
6664085    0.001644
3072459    0.001644
6694085    0.001644
1673501    0.001644
7926810    0.001644
504225     0.001644
             ...   
9418721    0.000183
7444861    0.000183
9466316    0.000183
1718737    0.000183
8622552    0.000183
3987933    0.000183
1184228    0.000183
7414258    0.000183
7014899    0.000183
4176261    0.000183
3664247    0.000183
2572502    0.000183
5777679    0.000183
1607899    0.000183
9814246    0.000183
3344635    0.000183
219389     0.000183
3518720    0.000183
2474246    0.000183


Number of unique SKU's in IL locations of the brand Polo Jeans Co. and/or in department Ralph Lauren = 2248

# Association Rules:

Grouping unique transactions together and one-hot-encoding:

In [12]:
basket = (trnsact_il
          .groupby(['id', 'SKU'])['QUANTITY']
          .sum().unstack().reset_index().fillna(0)
          .set_index('id'))

In [341]:
basket.head()

SKU,4072,11563,13388,14223,17934,18483,24072,24223,27934,28483,...,9884256,9891638,9901638,9914634,9917933,9941786,9957916,9974071,9991786,9994222
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
basket2 = pd.DataFrame(basket.values)
basket2.columns = list(basket.columns)
basket2.index = list(basket.index)
basket2 = basket2.astype(int)
basket2.shape

(3676, 2248)

In [14]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket2.applymap(encode_units)
basket_sets.shape

(3676, 2248)

In [348]:
basket_sets.head()

,4072,11563,13388,14223,17934,18483,24072,24223,27934,28483,...,9884256,9891638,9901638,9914634,9917933,9941786,9957916,9974071,9991786,9994222
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We want to look at items that appear in more than 1 transaction. A minimum support of 0.05% of 3672 transactions means that the items appear in at least 2 transactions. We can use the same support for every item here because all items are already filtered together and are very similar.

In [15]:
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [321]:
frequent_itemsets

,support,itemsets
0,0.001089,(4072)
1,0.000817,(14223)
2,0.000545,(18483)
3,0.000817,(24072)
4,0.000545,(24223)
5,0.000545,(28483)
6,0.001362,(34223)
7,0.001089,(38483)
8,0.001362,(44223)
9,0.001362,(44635)


In [55]:
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [56]:
#frequent_itemsets[frequent_itemsets['length'] >= 2].shape

(0, 3)

In [16]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=10)
rules.antecedents.nunique()

123

In [21]:
rules.sort_values(by=["confidence"],ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
61,(4823869),(3263869),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf
90,(5353003),(5773003),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf
60,(3263869),(4823869),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf
125,(9294395),(9324395),0.000545,0.001089,0.000545,1.000000,918.000000,0.000544,inf
24,(1324228),(6184085),0.000545,0.001634,0.000545,1.000000,612.000000,0.000544,inf
86,(6777933),(4917933),0.000545,0.001634,0.000545,1.000000,612.000000,0.000544,inf
57,(3148046),(6837616),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf
120,(8632552),(9182555),0.000545,0.001089,0.000545,1.000000,918.000000,0.000544,inf
56,(6837616),(3148046),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf
91,(5773003),(5353003),0.000545,0.000545,0.000545,1.000000,1836.000000,0.000544,inf


In [21]:
(rules[rules.confidence == 1].shape[0])/rules.shape[0]

0.23943661971830985

In [24]:
(rules[rules.confidence == 0.5].shape[0])/rules.shape[0]

0.15492957746478872

In [23]:
(rules[rules.confidence <= 0.333333].shape[0])/rules.shape[0]

0.19014084507042253

In [25]:
(rules[rules.confidence <= 0.2].shape[0])/rules.shape[0]

0.07042253521126761

Criteria: confidence above 0.2

We keep only the rules with a confidence above 0.2.

In [17]:
rules = rules[rules.confidence > .2].sort_values(by=["lift"],ascending = False)

In [28]:
rules[:31]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
91,(5773003),(5353003),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
52,(3074240),(3904240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
62,(4437898),(3287898),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
61,(4823869),(3263869),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
60,(3263869),(4823869),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
57,(3148046),(6837616),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
56,(6837616),(3148046),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
90,(5353003),(5773003),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
53,(3904240),(3074240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
51,(3064240),(3894240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf


In [29]:
rules[31:63]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,(2652631),(4662914),0.001089,0.000545,0.000545,0.500000,918.0,0.000544,1.998911
124,(9324395),(9294395),0.001089,0.000545,0.000545,0.500000,918.0,0.000544,1.998911
42,(4662914),(2652631),0.000545,0.001089,0.000545,1.000000,918.0,0.000544,inf
140,(9238597),"(4568105, 7617591)",0.001089,0.000545,0.000545,0.500000,918.0,0.000544,1.998911
137,"(4568105, 7617591)",(9238597),0.000545,0.001089,0.000545,1.000000,918.0,0.000544,inf
74,(4568105),(7617591),0.000817,0.000817,0.000545,0.666667,816.0,0.000544,2.997549
75,(7617591),(4568105),0.000817,0.000817,0.000545,0.666667,816.0,0.000544,2.997549
73,(6337594),(4537594),0.000817,0.000817,0.000545,0.666667,816.0,0.000544,2.997549
72,(4537594),(6337594),0.000817,0.000817,0.000545,0.666667,816.0,0.000544,2.997549
107,(6554238),(8224240),0.000817,0.000817,0.000545,0.666667,816.0,0.000544,2.997549


In [30]:
rules[63:94]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(9146222),(1056223),0.001089,0.000817,0.000545,0.500000,612.000000,0.000544,1.998366
131,"(7046264, 1814220)",(1824220),0.000545,0.001906,0.000545,1.000000,524.571429,0.000544,inf
134,(1824220),"(7046264, 1814220)",0.001906,0.000545,0.000545,0.285714,524.571429,0.000544,1.399237
68,(4080379),(8060116),0.001906,0.000817,0.000817,0.428571,524.571429,0.000815,1.748570
69,(8060116),(4080379),0.000817,0.001906,0.000817,1.000000,524.571429,0.000815,inf
84,(4822018),(4862018),0.000817,0.001362,0.000545,0.666667,489.600000,0.000544,2.995915
85,(4862018),(4822018),0.001362,0.000817,0.000545,0.400000,489.600000,0.000544,1.665305
127,(9432506),(9442506),0.001089,0.001089,0.000545,0.500000,459.000000,0.000543,1.997821
126,(9442506),(9432506),0.001089,0.001089,0.000545,0.500000,459.000000,0.000543,1.997821
119,(9182555),(8382552),0.001089,0.001089,0.000545,0.500000,459.000000,0.000543,1.997821


In [55]:
print("number of rules with confidence > 0.2:" ,rules.shape[0])

number of rules: 132


In [18]:
def top_100_skus(antecendents):
    best_skus = []
    for frozen_set in antecendents:
        if len(best_skus) == 100:
            return best_skus
        for sku in frozen_set:
            if sku in best_skus:
                continue
            else:
                best_skus.append(sku)

top100 = top_100_skus(rules.antecedents)

In [19]:
print("The 100 SKUs that are the best candidates to modify the planograms with are:\n",top100)

The 100 SKUs that are the best candidates to modify the planograms with are:
 [5773003, 3074240, 4437898, 4823869, 3263869, 3148046, 6837616, 5353003, 3904240, 3064240, 3894240, 306827, 8686826, 3287898, 5454240, 8234240, 5906249, 5666249, 7617591, 304072, 4568105, 9238597, 2324085, 2864398, 632916, 9294395, 9182555, 3986264, 456264, 8632552, 2652631, 9324395, 4662914, 6337594, 4537594, 6554238, 6684219, 8224240, 7046264, 6034219, 1814220, 1824220, 6372524, 5452548, 1324228, 6777933, 4917933, 1056223, 9068481, 5677937, 6184085, 3396339, 336339, 9146222, 4080379, 8060116, 4862018, 4822018, 9432506, 9442506, 8382552, 4256463, 9296463, 1897917, 6337897, 574072, 2484725, 2264226, 2934275, 6564238, 1824225, 5914721, 954722, 984225, 7784238, 4618105, 9797919, 9721644, 7926810, 8953989, 2870559, 6487933, 4637933, 4090379, 8784749, 6984275, 2397933, 8783989, 9734273, 2307933, 1144228, 2340559, 3907933, 3897933, 5036463, 294072, 2314085, 7727933, 8437933, 3204072]


In [49]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
91,(5773003),(5353003),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
52,(3074240),(3904240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
62,(4437898),(3287898),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
61,(4823869),(3263869),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
60,(3263869),(4823869),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
57,(3148046),(6837616),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
56,(6837616),(3148046),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
90,(5353003),(5773003),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
53,(3904240),(3074240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf
51,(3064240),(3894240),0.000545,0.000545,0.000545,1.000000,1836.0,0.000544,inf


In [19]:
top20 = top100[:20]
print("Top 20 moves to planograms:")
print(top20)

Top 20 moves to planograms:
[5773003, 3074240, 4437898, 4823869, 3263869, 3148046, 6837616, 5353003, 3904240, 3064240, 3894240, 306827, 8686826, 3287898, 5454240, 8234240, 5906249, 5666249, 7617591, 304072]


Comparing antecedents and consequents to see whether both Polo Jeans Co. brands and Ralph Lauren dept clothes are in the same rule:

If many transactions have both brand Polo Jeans Co. and Ralph Lauren department items, we can put these items in the same departments or at least put the separate departments close to each other.

If not, then we can keep them in separate departments but items with associations should be near each other within the same department.

In [63]:
count = 0
ants = list(rules.antecedents)
cons = list(rules.consequents)
for i in range(100):
    for sku_ant in ants[i]:
        for sku_con in cons[i]:
            if sku_ant in polo and sku_con in rlauren:
                count += 1
            elif sku_ant in rlauren and sku_con in polo:
                count += 1
        
print(count)

0


None of the association rules involve both Polo Jeans Co. and Ralph Lauren items in the same rule.

It doesn't matter that the two brands are subsidiaries of the same parent company. They should be kept in separate departments.

Customers who wear Polo Jeans Co. clothes may not like Ralph Lauren attire and vice versa. This may be that although they share the same parent company, they are viewed as completely different styles.

# Looking at associating styles, colors, brands:

In [59]:
ants = []
for f_set in list(rules.antecedents)[:20]:
    for a in f_set:
        ants.append(a)

print(ants)
        
cons = []
for f_set in list(rules.consequents)[:20]:
    for c in f_set:
        cons.append(c)
    
print(cons)

[5773003, 3074240, 4437898, 4823869, 3263869, 3148046, 6837616, 5353003, 3904240, 3064240, 3894240, 306827, 8686826, 3287898, 5454240, 8234240, 5906249, 5666249, 7617591, 304072]
[5353003, 3904240, 3287898, 3263869, 4823869, 6837616, 3148046, 5773003, 3074240, 3894240, 3064240, 8686826, 306827, 4437898, 8234240, 5454240, 5666249, 5906249, 4568105, 9238597, 2324085]


In [119]:
a_styles = pd.DataFrame(ants)
a_styles.columns = ["SKU_PK"]
a_styles = a_styles.merge(data_sku, how='left')
a_styles = a_styles.drop(columns=["DEPT_FK","CLASSID","UPC","PACKSIZE","VENDOR"])
a_styles.columns = ["antecedents","STYLE","COLOR","SIZE","BRAND"]
a_styles = a_styles[:-1].append(pd.Series(),ignore_index=True).append(a_styles[-1:])
a_styles = a_styles.reset_index(drop=True)
int_ants = []
for i in range(len(a_styles.antecedents)):
    if i != 19:
        int_ants.append(int(a_styles.antecedents[i]))
    else:
        int_ants.append(int_ants[-1])
a_styles.antecedents = int_ants

c_styles = pd.DataFrame(cons)
c_styles.columns = ["SKU_PK"]
c_styles = c_styles.merge(data_sku, how='left')
c_styles = c_styles.drop(columns=["DEPT_FK","CLASSID","UPC","PACKSIZE","VENDOR"])
c_styles.columns = ["consequents","STYLE","COLOR","SIZE","BRAND"]
c_styles

pd.concat([a_styles,c_styles],axis=1)

,antecedents,STYLE,COLOR,SIZE,BRAND,consequents,STYLE,COLOR,SIZE,BRAND
0,5773003,921641N70917,RED,12,TOMMY HI,5353003,579118N70223,SNOW WHITE,12,TOMMY HI
1,3074240,47 RG5427,882DEVONRE,24,RALPH LA,3904240,15 RG5427,210ANWT,24,RALPH LA
2,4437898,62 RJ1443,PALEROSE,18,RALPH LA,3287898,19 RJ1440,PINK,18,RALPH LA
3,4823869,97 RG8422,168FLORAL,9,RALPH LA,3263869,96 RG8422,168FLORAL,9,RALPH LA
4,3263869,96 RG8422,168FLORAL,9,RALPH LA,4823869,97 RG8422,168FLORAL,9,RALPH LA
5,3148046,55 BG5428,110WHITE,12,RALPH LA,6837616,55 BG5428,908BEACHRY,12,RALPH LA
6,6837616,55 BG5428,908BEACHRY,12,RALPH LA,3148046,55 BG5428,110WHITE,12,RALPH LA
7,5353003,579118N70223,SNOW WHITE,12,TOMMY HI,5773003,921641N70917,RED,12,TOMMY HI
8,3904240,15 RG5427,210ANWT,24,RALPH LA,3074240,47 RG5427,882DEVONRE,24,RALPH LA
9,3064240,47 RG5427,882DEVONRE,18,RALPH LA,3894240,15 RG5427,210ANWT,18,RALPH LA


All antecedent and consequent items in the 20 rules with the highest lift metric have the same size and brand. It is clear that customers that buy one article of clothing would buy another of the same size and brand.